<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering/blob/main/11_Evaluating_Numerical_Expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating Numerical Expressions

In this project, we'll use stacks to build a function that can evaluate complex numerical expressions stored within a string.

For example, to calculate the expression `21 / (11 - 4) * 2 + 5`, we'll implement a function named `evaluate()` to evaluate the expression stored as a string.

    expression = "21 / (11 - 4) * 2 + 5"
    print(evaluate(expression))
    11

First we'll define a class called `Stack` that will be used to create linked lists and stack the individual elements together from the expression to be evaluated.

In [ ]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [ ]:
class Stack:

    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1

    def __iter__(self):
        self._iter_node = self.head
        return self 

    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret

    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str([value for value in self])
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Infix & Postfix Notation

Expression are written using infix notation, meaning that the operators are placed between the operands. For example, the operator + is between two operands in the expression `1 + 10`.

In postfix notation, the operands come before the operator, so `1 + 10` would look like `1 10 +`. This notation is easier for a computer to understand, and we can evaluate an expression in postfix using a stack. 

By reading the expression from left to right, if there is a number, that number gets pushed to the top of the stack. Then when we find an operator, we pop the top two elements of the stack, perform than operation, and push the result back into the stack. In the end, we're left with one last final value that is the result of our expression. 

For our function to work, we're going to evaluate in postfix notation. We can then later on create a function to convert an expression in infix notation into postfix to be more user-friendly. The first step is to transform our expression into a list of tokens.

In [ ]:
def tokenize(expression):
    return expression.split()

print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


## Processing an Operator

When our function comes across an operator, we need it to pop the top two values of the stack, and apply the operator to those two elements in the correct order. So if our expression is `10 - 1`, we need to subtract 1 from 10 and not 10 from 1. The element at the top of the stack is the 2nd operand in the expression, and the 2nd element in the stack is the 1st operand.

We'll define several functions for to account for each of the operators we'll find in our expressions.

In [ ]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top  # subtraction
    stack.push(result)

In [ ]:
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top # addition
    stack.push(result)

In [ ]:
def process_time(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top # multiplication
    stack.push(result)

In [ ]:
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top # division
    stack.push(result)

In [ ]:
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top # exponential
    stack.push(result)

## Evaluating Postfix Expressions

Next, we'll create a function called `evaluate_postfix()` that will tokenize our expression, and then extract the operator and call the corresponding function to process the expression.

In [ ]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == '-':
            process_minus(stack)
        elif token == '+':
            process_plus(stack)
        elif token == '*':
            process_time(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

In [ ]:
# Testing the function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

Now that we've enabled the algorithm to evaluate postfix expressions, we need to take it one step further to enable it to evaluate infix expressions. We can do this by implementing a [Shunting-yard algorithm](https://en.wikipedia.org/wiki/Shunting-yard_algorithm#:~:text=In%20computer%20science%2C%20the%20shunting,abstract%20syntax%20tree%20(AST).) and  create a dictionary of precedence.

The operation precedence rules are very important because they define the order in which the operations are performed. For example, in the expression `2 + 3 * 4`, we first need to multiply 3 by 4 before we add the 2. We can compare the precedence of our operators in our dictionary by assigning number values to the operators. This makes it easier to check if an operator has higher or lower precedence than another.

In [ ]:
precedence = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2,
    '**': 3
}

# Test the precedence dictionary
print(precedence['+'] < precedence['*'])
print(precedence['-'] < precedence['+'])
print(precedence['/'] < precedence['**'])
print(precedence['**'] < precedence['-'])

True
False
True
False


## Handling Parenthesis

The next challenge in evaluating expressions is dealing with parenthesis.

For **opening parenthesis**, we'll need to call a function that will push the string "(" into the stack.

For **closing parenthesis**, as long as the top of the stack isn't an opening parenthesis, we'll need a function that pops the top element and appends it to the postfix token list. The last part of our function will pop the opening parentheses out from the end of the stack.

In [ ]:
# Opening Parenthesis
def open_parenthesis(stack):
    stack.push('(')

# Closing Parenthesis
def close_parenthesis(stack, postfix):
    while stack.peek() != '(':
        postfix.append(stack.pop()) # Adds tokens until we reach "("
    stack.pop() # Removes the "(" from the end of the stack

## Handling Operators

We need a function to handle the operators in the expression. This function will pop the top element and append it to the postfix token list as long as the top of the stack is an operator with a precedence greater than or equal to the current operator. Then it will push the current operator to the top of the stack.

`stack.peek()` will allow us to view the top of the stack without removing it.

In [ ]:
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)

## Handling Operands

The last function to process the tokens in our expression will need to handle the numbers. Any number we come across will need to be pushed into the postfix token list.

In [ ]:
def process_number(postfix, number):
    postfix.append(number)

## Implementing the Shunting-yard Algorithm

Now we have everything we need to convert an expression from infix notation to postfix notation. The function will work like this:

1. Split the expression into tokens using `tokenize()`.
2. Initialize an empty stack.
3. Initialize an empty postfix token list.
4. Iterate over all tokens, and for each token:
    * If token is "(", call `open_parenthesis()`
    * If token is ")", call `close_parenthesis()`
    * If token is operator, call `process_operator()`
    * If token is operand, call `process_number()`
5. After all tokens are processed, pop the remaining stack element into the postfix token list.
6. Convert the postfix token into a string using `str.join()`

In [ ]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == '(':
            open_parenthesis(stack)
        elif token == ')':
            close_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return ' '.join(postfix)

## Evaluating Infix Expressions

Now we'll combine the function that transforms infix expressions to postfix notation and the function that evaluates that expression in postfix notation into a function called `evaluate()`.

In [ ]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

## Testing the Function

Let's test the function on some expressions and see if we get the correct results.

In [ ]:
expressions = [
    '1 + 2',
    '1 * ( 2 - ( 1 + 2 ) )',
    '4 * ( 1 + 2 * ( 9 / 3 ) - 5 )',
    '10 + 3 * 5 / ( 16 - 4 * 1 )',
    '2 * 2 * 2 * 2 * 2 * 2 * 2 * 2',
    '2 ** 2 ** 2 ** 2 ** 2',
    '( 1 - 2 ) / ( 3 - 5 )',
    '9 / 8 * 8',
    '64 / ( 8 * 8 )',
]

for expression in expressions:
    print(evaluate(expression))

3.0
-1.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0


## Conclusion & Next Steps

In this project we created a function to evaluate expressions stored as strings. This function shows us just how powerful stacks can be and how they can be used to help us solve complex problems. 

Some next steps we could take to make our function more robust are:
* Improve `tokenize()` so that functions can be written without a space between each item in the string.
* Add more functionality like a square root operator or integer division. 

The idea for this project comes from the [DATAQUEST](https://app.dataquest.io/) **Data Structures Fundamentals** course.